In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import os
import warnings

from dotenv import load_dotenv

warnings.filterwarnings("ignore")

In [30]:
from app.utils.data_loader import DataLoader
from app.utils.health_score import HealthScore
from app.utils.portfolios import Portfolios
from app.utils.portfolios_repo import PortfoliosRepository
from app.utils.portfolios_service import PortfolioService
from app.utils.portprop_matrices import PortpropMatrices
from app.utils.portprop_matrices_repo import PortpropMatricesRepository
from app.utils.rebalancer import Rebalancer
from app.utils.rebalancer_repo import RebalancerRepository

## Load Data

In [31]:
load_dotenv()  # Load environment variables from .env file
print(os.getenv("LOAD_DATA_FROM_DWH"))

false


In [32]:
data_loader = DataLoader(load_from_db=False)
ppm_repo = PortpropMatricesRepository(data_loader=data_loader)
ports_repo = PortfoliosRepository(data_loader=data_loader)
rebalancer_repo = RebalancerRepository(data_loader=data_loader)

In [33]:
client_out_enriched = ports_repo.load_client_out_product_enriched(
    as_of_date="2025-09-30"
)
client_styles = ports_repo.load_client_style(as_of_date="2025-09-30")

In [34]:
ports_ref_table = {
    "product_mapping": ports_repo.load_product_mapping(),
    "product_underlying": ports_repo.load_product_underlying(),
}

In [35]:
ppm_ref_dict = {
    "portprop_factsheet": ppm_repo.load_portprop_factsheet(),
    "portprop_benchmark": ppm_repo.load_portprop_benchmark(),
    "portprop_ge_mapping": ppm_repo.load_portprop_ge_mapping(),
    "portprop_fallback": ppm_repo.load_portprop_fallback(),
    "portprop_ret_eow": ppm_repo.load_portprop_ret_eow(),
    "advisory_health_score": ppm_repo.load_advisory_health_score(),
}

In [36]:
rb_ref_dict = {
    "es_sell_list": rebalancer_repo.load_es_sell_list(),
    "product_recommendation_rank_raw": rebalancer_repo.load_product_recommendation_rank_raw(),
    "mandate_allocation": rebalancer_repo.load_mandate_candidates(),
}

## Instances

In [37]:
## Portsfolios
ports_all = Portfolios()
ports_all.set_ref_tables(ports_ref_table)
df_out, df_style, port_ids, port_id_mapping = ports_all.create_portfolio_id(
    client_out_enriched, client_styles, column_mapping=["as_of_date", "customer_id"]
)
ports_all.set_portfolio(df_out, df_style, port_ids, port_id_mapping)

## Portfolio Service
port_service = PortfolioService(ports_all)

## Portprop Matrices
ppm = PortpropMatrices(ppm_ref_dict)

## Health Score
hs = HealthScore()

## Rebalancer
rb = Rebalancer(
    client_investment_style="Moderate High Risk",
    client_classification="UI",
    discretionary_acceptance=0.2,
    new_money=1_000_000,
    product_whitelist=["KKP", "PTTEP"],
    product_blacklist=["KKP GNP", "K-GSELECTU-A(A)"],
)
rb.set_ref_tables(rb_ref_dict)

## Portfolio (Service)

In [55]:
## get list of all customer ids
print(port_service.get_all_customer_ids()[0:5])

[26527, 25914, 31882, 110343, 24191]


In [39]:
## get single port from customer id
port = port_service.get_client_portfolio(customer_id=21105)

In [40]:
port.df_out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 114528 to 132008
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           19 non-null     int64         
 1   as_of_date            19 non-null     datetime64[ns]
 2   product_id            19 non-null     object        
 3   src_sharecodes        19 non-null     object        
 4   desk                  19 non-null     object        
 5   port_type             19 non-null     object        
 6   currency              19 non-null     string        
 7   product_display_name  19 non-null     string        
 8   product_type_desc     19 non-null     string        
 9   asset_class_name      19 non-null     string        
 10  symbol                19 non-null     string        
 11  pp_asset_sub_class    19 non-null     string        
 12  is_risky_asset        19 non-null     bool          
 13  coverage_prdtype  

In [41]:
## get port outstanding
port.df_out

,customer_id,as_of_date,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,...,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,value,port_id,asset_class_code,weight
114528,21105,2025-09-30,S00088794,SCCC,TRADE,L,THB,SCCC,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,432100.00,4962,AA_LE,0.120891
116232,21105,2025-09-30,S00080400,BTS,TRADE,L,THB,BTS,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,10208.16,4962,AA_LE,0.002856
117184,21105,2025-09-30,S00080320,TTA,TRADE,L,THB,TTA,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,34128.00,4962,AA_LE,0.009548
118167,21105,2025-09-30,S00086119,KKP,TRADE,L,THB,KKP,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,264375.00,4962,AA_LE,0.073966
118176,21105,2025-09-30,S00088866,ASP,TRADE,L,THB,ASP,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,110700.00,4962,AA_LE,0.030971
118998,21105,2025-09-30,S00080094,BBL,TRADE,L,THB,BBL,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,175820.00,4962,AA_LE,0.049190
119932,21105,2025-09-30,S00080422,ADVANC,TRADE,L,THB,ADVANC,Listed Securities,Local Equity,...,True,0.042,False,<NA>,Not Top-Pick,<NA>,1280400.00,4962,AA_LE,0.358224
119945,21105,2025-09-30,S00083319,DCC,TRADE,L,THB,DCC,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,234600.00,4962,AA_LE,0.065635
120799,21105,2025-09-30,S00087467,BTS-W8,TRADE,L,THB,BTS-W8,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,13.34,4962,AA_LE,0.000004
121655,21105,2025-09-30,S00080158,TTB,TRADE,L,THB,TTB,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,48227.70,4962,AA_LE,0.013493


In [42]:
port.product_mapping.columns

Index(['product_id', 'src_sharecodes', 'desk', 'port_type', 'currency',
       'product_display_name', 'product_type_desc', 'asset_class_name',
       'symbol', 'pp_asset_sub_class', 'is_risky_asset', 'coverage_prdtype',
       'is_coverage', 'expected_return', 'es_core_port', 'es_sell_list',
       'flag_top_pick', 'flag_tax_saving'],
      dtype='object')

In [43]:
# get port allocation lookthrough
port.get_portfolio_asset_allocation_lookthrough(ppm)

asset_class,port_id,aa_alt,aa_cash,aa_fi,aa_ge,aa_le
0,4962,0.046886,0.091343,0,0,0.861771


In [44]:
# get model allocation
port.get_model_asset_allocation_lookthrough(ppm)

,port_id,port_investment_style,portpop_styles,aa_alt_model,aa_cash_model,aa_fi_model,aa_ge_model,aa_le_model
0,4962,Aggressive Growth,Aggressive,0.105,0.055,0.08,0.684,0.076


In [45]:
port.df_style

,port_id,port_investment_style,portpop_styles
20768,4962,Aggressive Growth,Aggressive


## Healthscore

In [46]:
## get client health score
health_score, health_score_comp = port.get_portfolio_health_score(ppm, hs)

In [47]:
health_score["health_score"].values[0]

np.float64(4.0)

In [48]:
health_score_comp

,port_id,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,value,...,ge_other,expected_return,volatility,is_bulk_risk,underlying_company,issure_risk_group,coverage_prdtype,score_non_cover_global_stock,score_non_cover_local_stock,score_non_cover_mutual_fund
0,4962,S00088794,SCCC,TRADE,L,THB,SCCC,Listed Securities,Local Equity,432100.00,...,NaN,5.077421e-03,2.206245e-02,False,SCCC,NaN,LOCAL_STOCK,0,-1,0
1,4962,S00080400,BTS,TRADE,L,THB,BTS,Listed Securities,Local Equity,10208.16,...,NaN,1.199517e-04,5.212150e-04,False,BTS,NaN,LOCAL_STOCK,0,-1,0
2,4962,S00080320,TTA,TRADE,L,THB,TTA,Listed Securities,Local Equity,34128.00,...,NaN,4.010234e-04,1.742530e-03,False,TTA,NaN,LOCAL_STOCK,0,-1,0
3,4962,S00086119,KKP,TRADE,L,THB,KKP,Listed Securities,Local Equity,264375.00,...,NaN,3.106557e-03,1.349863e-02,False,KKP,NaN,LOCAL_STOCK,0,-1,0
4,4962,S00088866,ASP,TRADE,L,THB,ASP,Listed Securities,Local Equity,110700.00,...,NaN,1.300788e-03,5.652194e-03,False,ASP,NaN,LOCAL_STOCK,0,-1,0
5,4962,S00080094,BBL,TRADE,L,THB,BBL,Listed Securities,Local Equity,175820.00,...,NaN,2.065985e-03,8.977134e-03,False,BBL,NaN,LOCAL_STOCK,0,-1,0
6,4962,S00080422,ADVANC,TRADE,L,THB,ADVANC,Listed Securities,Local Equity,1280400.00,...,NaN,1.504543e-02,6.537551e-02,True,ADVANC,1.0,LOCAL_STOCK,0,0,0
7,4962,S00083319,DCC,TRADE,L,THB,DCC,Listed Securities,Local Equity,234600.00,...,NaN,2.756683e-03,1.197836e-02,False,DCC,NaN,LOCAL_STOCK,0,-1,0
8,4962,S00087467,BTS-W8,TRADE,L,THB,BTS-W8,Listed Securities,Local Equity,13.34,...,NaN,1.567526e-07,6.811225e-07,False,BTS,NaN,LOCAL_STOCK,0,-1,0
9,4962,S00080158,TTB,TRADE,L,THB,TTB,Listed Securities,Local Equity,48227.70,...,NaN,5.667029e-04,2.462442e-03,False,TTB,NaN,LOCAL_STOCK,0,-1,0


In [49]:
health_score_comp.columns

Index(['port_id', 'product_id', 'src_sharecodes', 'desk', 'port_type',
       'currency', 'product_display_name', 'product_type_desc',
       'asset_class_name', 'value', 'weight', 'aa_alt', 'aa_cash', 'aa_fi',
       'aa_ge', 'aa_le', 'ge_em', 'ge_eur', 'ge_jp', 'ge_us', 'ge_other',
       'expected_return', 'volatility', 'is_bulk_risk', 'underlying_company',
       'issure_risk_group', 'coverage_prdtype', 'score_non_cover_global_stock',
       'score_non_cover_local_stock', 'score_non_cover_mutual_fund'],
      dtype='object')

## Rebalancer

In [50]:
new_port, recommendations = rb.rebalance(port, ppm, hs)

[TEMP-DEBUG][rebalance] start | new_money= 1000000
[TEMP-DEBUG][sell] row below thresholds w_change=0.000004 a_abs=13.34
[TEMP-DEBUG][sell] row below thresholds w_change=0.000137 a_abs=491.25
[TEMP-DEBUG][sell] row below thresholds w_change=0.002856 a_abs=10208.16
[TEMP-DEBUG][sell] row below thresholds w_change=0.009548 a_abs=34128.00
[TEMP-DEBUG][sell] row below thresholds w_change=0.011695 a_abs=41800.00
[TEMP-DEBUG][sell] row below thresholds w_change=0.012730 a_abs=45500.00
[TEMP-DEBUG][sell] row below thresholds w_change=0.013493 a_abs=48227.70
[TEMP-DEBUG][sell] row below thresholds w_change=0.018719 a_abs=66908.95
[TEMP-DEBUG][sell] row below thresholds w_change=0.030971 a_abs=110700.00
[TEMP-DEBUG][sell] row below thresholds w_change=0.049190 a_abs=175820.00
[TEMP-DEBUG][sell] exception occurred: "['action'] not in index"
[TEMP-DEBUG][rebalance] sells rows= 0
[TEMP-DEBUG][rebalance] cash_shift rows= 0
[TEMP-DEBUG][rebalance] convert_ccy rows= 0


Traceback (most recent call last):
  File "/Users/home/projects/is-personalized-advisory/app/utils/rebalancer.py", line 35, in build_sell_recommendations
    product_whitelist: list[str] | None = None,
    
  File "/Users/home/projects/is-personalized-advisory/.venv/lib/python3.13/site-packages/pandas/core/frame.py", line 4108, in __getitem__
    indexer = self.columns._get_indexer_strict(key, "columns")[1]
              ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^
  File "/Users/home/projects/is-personalized-advisory/.venv/lib/python3.13/site-packages/pandas/core/indexes/base.py", line 6200, in _get_indexer_strict
    self._raise_if_missing(keyarr, indexer, axis_name)
    ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/home/projects/is-personalized-advisory/.venv/lib/python3.13/site-packages/pandas/core/indexes/base.py", line 6252, in _raise_if_missing
    raise KeyError(f"{not_found} not in index")
KeyError: "['action'] not in index"


[TEMP-DEBUG][rebalance] buys rows= 0
[TEMP-DEBUG][rebalance] final recommendations rows= 0
[TEMP-DEBUG][rebalance] final df_out rows= 19


In [51]:
recommendations

,transaction_no,batch_no,port_id,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,value,weight,flag,expected_weight,action,amount


In [52]:
new_port.df_out

,customer_id,as_of_date,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,...,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,value,port_id,asset_class_code,weight
114528,21105,2025-09-30,S00088794,SCCC,TRADE,L,THB,SCCC,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,432100.00,4962,AA_LE,0.120891
116232,21105,2025-09-30,S00080400,BTS,TRADE,L,THB,BTS,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,10208.16,4962,AA_LE,0.002856
117184,21105,2025-09-30,S00080320,TTA,TRADE,L,THB,TTA,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,34128.00,4962,AA_LE,0.009548
118167,21105,2025-09-30,S00086119,KKP,TRADE,L,THB,KKP,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,264375.00,4962,AA_LE,0.073966
118176,21105,2025-09-30,S00088866,ASP,TRADE,L,THB,ASP,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,110700.00,4962,AA_LE,0.030971
118998,21105,2025-09-30,S00080094,BBL,TRADE,L,THB,BBL,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,175820.00,4962,AA_LE,0.049190
119932,21105,2025-09-30,S00080422,ADVANC,TRADE,L,THB,ADVANC,Listed Securities,Local Equity,...,True,0.042,False,<NA>,Not Top-Pick,<NA>,1280400.00,4962,AA_LE,0.358224
119945,21105,2025-09-30,S00083319,DCC,TRADE,L,THB,DCC,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,234600.00,4962,AA_LE,0.065635
120799,21105,2025-09-30,S00087467,BTS-W8,TRADE,L,THB,BTS-W8,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,13.34,4962,AA_LE,0.000004
121655,21105,2025-09-30,S00080158,TTB,TRADE,L,THB,TTB,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,48227.70,4962,AA_LE,0.013493


In [53]:
health_score, health_score_comp = new_port.get_portfolio_health_score(ppm, hs)

In [54]:
health_score["health_score"].values[0]

np.float64(4.0)